<a href="https://colab.research.google.com/github/Alfred9/Natural-Language-Processing/blob/main/T5_LINGUISTICS/T5_LINGUISTICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyspark spark-nlp

In [2]:
import json
import pandas as pd
import numpy as np

import sparknlp
import pyspark.sql.functions as F

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [3]:
spark = sparknlp.start()

spark

In [4]:
documentAssembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

t5 = T5Transformer.pretrained("t5_grammar_error_corrector") \
    .setTask("gec:") \
    .setInputCols(["documents"])\
    .setMaxOutputLength(200)\
    .setOutputCol("corrections")

pipeline = Pipeline().setStages([documentAssembler,
                                 t5])


t5_grammar_error_corrector download started this may take some time.
Approximate size to download 883.7 MB
[OK!]


In [5]:
pipeline_model = pipeline.fit(spark.createDataFrame([['']]).toDF('text'))
T5model = LightPipeline(pipeline_model)

example_txt = """

Anna and Mike is going skiing and they is liked is
"""

res = T5model.fullAnnotate(example_txt)[0]


print ('Prediction:', res['corrections'][0].result)

Prediction: Anna and Mike are going skiing and they like it.
